In [ ]:
import os 
# Define the directory containing the dataset
directory = "FashionStyle14_v1/"

def rename_files(directory):
    """
    Rename files in the dataset directory by appending folder name and index.
    Args:
        directory (str): Path to the dataset directory.
    Returns:
        None
    """
    directory = os.path.join(directory, "dataset")
    # Iterate through each folder in the directory
    for folder in os.listdir(directory):
        # Skip system files like .DS_Store
        if folder == ".DS_Store":
            continue
        # Iterate through each file in the folder
        for i, file in enumerate(os.listdir(os.path.join(directory, folder))):
            # Skip hidden files
            if not file.startswith('.'):
                # Create a new name using the folder name and index
                new_name = f"{folder}_{i}.jpg"
                # Rename the file
                os.rename(os.path.join(directory, folder, file), os.path.join(directory, folder, new_name))


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

def label_dataset(directory):
    """
    Create and save a dataset.
    Save label encoder classes to a numpy file.
    Args:
        directory (str): Path to the directory containing images and labels.
    Returns:
        None
    """
    labels = ['conservative', 'dressy', 'feminine', 'kireime-casual', 'mode', 'natural', 'retro',
       'rock', 'street']
    images_directory = os.path.join(directory, 'dataset')
    df = pd.DataFrame(columns=['image_path'])
    for folder in os.listdir(images_directory):
        if os.path.isdir(os.path.join(images_directory, folder)) and folder != ".DS_Store" and folder in labels:
            for file in os.listdir(os.path.join(images_directory, folder)):
                if file.startswith('.'):
                    continue
                file_path = os.path.join(folder, file)
                df = pd.concat([df, pd.DataFrame([[file_path]], columns=['image_path'])], ignore_index=True)

    # Extract labels from paths
    df.drop(df[df['image_path'] == ""].index, inplace=True)
    df['label'] = df['image_path'].apply(lambda x: x.split('/')[0])
    encoder = LabelEncoder()
    df['label'] = encoder.fit_transform(df['label'])
    # Save the merged dataframe to a CSV file
    df.to_csv(os.path.join(directory, 'dataset.csv'), index=False)
    np.save(os.path.join(directory, 'classes.npy'), encoder.classes_)
    return encoder.classes_

if "dataset.csv" in os.listdir(directory):
    os.remove(os.path.join(directory, "dataset.csv"))
label_dataset(directory)

**Visualizing data**

In [ ]:
import matplotlib.pyplot as plt

def visualize(labels, directory, csv_file, rows=8):
    """
    Visualize images from the dataset for each label.
    Args:
        labels (list): List of class labels.
        directory (str): Path to the dataset directory.
        csv_file (str): Name of the CSV file containing image paths and labels.
        rows (int): Number of rows of images to display for each label.
    Returns:
        None
    """
    fig, axes = plt.subplots(rows, len(labels), figsize=(20, 10))   
    # Load the dataset from the CSV file
    dataset = pd.read_csv(os.path.join(directory, csv_file))
    # Iterate through each label
    for i, label in enumerate(labels):
        # Filter the dataset for the current label
        label_data = dataset[dataset['label'] == i]
        for j in range(rows):
            image_path = os.path.join(directory, "dataset", label_data.iloc[j]['image_path'])
            # Read and display the image
            img = plt.imread(image_path)
            axes[j, i].imshow(img)
            axes[j, i].axis('off')  # Hide axes
            if j == 0:
                axes[j, i].set_title(label)
    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

# Load class names from the saved numpy file
class_names = np.load(os.path.join(directory, 'classes.npy'), allow_pickle=True).tolist()
# Visualize the dataset with 3 rows of images per label
visualize(class_names, directory, "dataset.csv", 3)
